# セキュリティ記事をコサイン類似度に基づきピックアップする

In [59]:
# japanize_matplotlibが未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install japanize-matplotlib

In [60]:
# janomeが未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install janome

In [61]:
# Googleドライブがマウントされていない場合に、2行のコメント解除して本セルを単独で実行する
#from google.colab import drive
#drive.mount('/content/drive')

In [62]:
# matplotlibで日本語を使えるようにする
import japanize_matplotlib

In [63]:
import pandas as pd
import numpy as np

In [64]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Tokenizerインスタンスを生成し、読み捨てる品詞を指定する

In [65]:
tokenizer = Tokenizer()
# 読み捨てるトークンの品詞を指定する
token_filters = [POSStopFilter(['記号','助詞','助動詞','動詞'])]
anal = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

## データファイルを読み込む

In [66]:
 # 列'header','body'を持つ記事テキストのcsvファイルを読み込む
 df1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Biglobe_articles.csv',encoding='cp932')
 df2 = df1.loc[:,['header','body']]
 df2.head(3)

,header,body
0,Zoomクライアントに2件の脆弱性、最新版へのアップデートを,写真を拡大 Zoom Video Communicationsは11月24日(米国時間...
1,TP-LINKのWi-Fiルータ「Enterprise AX1800」に任意のコード実行の脆弱性,写真を拡大 Security Affairsは11月26日、「Resecurity d...
2,トレンドマイクロの「ウイルスバスター for Mac」に特権昇格の脆弱性,写真を拡大 JPCERTコーディネーションセンター（JPCERT/CC: Japan ...


## 対象テキスト文字列を取り出し、分かち書き文を作る

In [67]:
df2 = df2.assign(wakati='')
# 対象テキスト文字列を一行ずつ取り出して分かち書きに変換する
for i in range(len(df2)):
  texts_flat = df2.iloc[i,1]
  tokens = anal.analyze(texts_flat)
  df2['wakati'][i] = ' '.join([t.surface for t in tokens])
df3 = df2.dropna(how='all')
df3.head(3)

,header,body,wakati
0,Zoomクライアントに2件の脆弱性、最新版へのアップデートを,写真を拡大 Zoom Video Communicationsは11月24日(米国時間...,写真 拡大 Zoom Video Communications 11 月 24 日 ( 米国...
1,TP-LINKのWi-Fiルータ「Enterprise AX1800」に任意のコード実行の脆弱性,写真を拡大 Security Affairsは11月26日、「Resecurity d...,写真 拡大 Security Affairs 11 月 26 日 Resecurity di...
2,トレンドマイクロの「ウイルスバスター for Mac」に特権昇格の脆弱性,写真を拡大 JPCERTコーディネーションセンター（JPCERT/CC: Japan ...,写真 拡大 JPCERT コーディネーション センター JPCERT / CC : Japa...


## 類似度評価の準備をする

In [68]:
# コサイン類似度のしきい値を定める
sim_thresh = 0.05
# 分かち書きをリストに、ヘッダと本文をNumpy配列に格納する
corpus = df3['wakati'].tolist()
header = np.array(df3['header'].tolist())
body = np.array(df3['body'].tolist())
# ユーザの利用しているアプリケーションを記述した文書を指定する
user_app = ['Google Chrome Microsoft Excel PowerPoint Python Anaconda SAKURA Editor Adobe Acrobat 脆弱性']

## ターゲット文書とのコサイン類似度を求め、近い文書を取り出す

In [69]:
# 結果から取り除くエスケープ文字を指定する
bad_chars = ["\n","\t","\r"]
# ユーザアプリの文書をニュース記事リストの先頭に挿入する
docs = user_app + corpus
# TF-IDFベクトル化する
vectorizer = TfidfVectorizer(max_df=0.9)
X = vectorizer.fit_transform(docs)
# コサイン類似度を計算する
sim = cosine_similarity(X)
# ターゲット文書に関わる一行目だけ残し、最初の要素は自分との比較なので取り除く
simil = sim[0][1:]
# コサイン類似度がしきい値を超えるインデックスを取得する。np.where結果はtuple
relev_index = np.where(simil > sim_thresh)[0]
# 取得したインデックスでヘッダと本文を抽出する
relev_head = header[relev_index]
print(relev_head)
relev_body = body[relev_index]
relev_body_r = []
for s in relev_body:
  relev_body_r.append(''.join((filter(lambda i: i not in bad_chars, s))))
print(relev_body_r)

['Windows 10、11などのゼロデイ脆弱性、パッチ公開前に報告者が概念実証コード公開'
 'Windowsの新しい特権昇格の脆弱性に関する概念実証コードが公開される'
 'Microsoft Exchange Serverの脆弱性に関する概念実証コード公開']
["  Microsoftによる「CVE-2021-41379」の説明写真を拡大   \u3000米Microsoftが11月のセキュリティ更新プログラムで修正したはずのWindowsインストーラのローカル昇格特権の脆弱性「CVE-2021-41379」はまだ完全に修正されておらず、依然として悪用される危険があると、この脆弱性をMicrosoftに報告した研究者が11月21日（現地時間）、GitHubに概念実証コードを公開した。\u3000米Cisco Systemsのセキュリティ部門Cisco Talos Intelligence Groupは23日、このゼロデイ脆弱性を悪用する実際のマルウェアサンプルを既に検出していると発表した。\u3000この脆弱性を報告した自営のセキュリティ研究者、アブデルハミド・ナセリ氏は、Microsoftが公開したCVE-2021-41379のパッチを分析したところ、バグが正しく修正されていないことを発見したという。    googletag.cmd.push(function() { googletag.display('div-gpt-ad-1599212539073-0'); });  \u3000同氏が公開した概念実証では、最新のパッチを当てたWindows 10、11、Windows Server 2022で、Microsoft Edge Elevation Service DACL（任意のアクセス制御リスト）を上書きし、それ自体をサービスの場所にコピーして実行し、昇格された特権を取得できたとしている。\u3000ナセリ氏は米Bleeping Computerに対し、この脆弱性の修正を待たずに概念実証を公開した理由を、Microsoftがバグ報奨金を減額したことに対する不満だと語った。\u3000本稿執筆現在、このリスクを軽減するための公式の回避策はまだない。ユーザーにとっての最善策はMicrosoftが正しく機能する新たなパッチを提供するのを待つことだとナセ